# **DECONVOLUTION**

In [17]:
from __future__ import print_function
import os
import cv2
import histomicstk as htk

import numpy as np
import scipy as sp

import skimage.io
import skimage.measure
import skimage.color

import time
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline

#Some nice default configuration for plots
plt.rcParams['figure.figsize'] = 15, 15
plt.rcParams['image.cmap'] = 'gray'
titlesize = 24

import copy
from tqdm import tqdm

In [18]:
base_img_path = 'val/'
images = os.listdir(base_img_path)


***Approach-1*** - Supervised color deconvolution with a known stain matrix

In [19]:
def normalize(stain_OD):
    normal_OD = []
    for stain in stain_OD:
        print(stain)
        deno = 0
        pow = [e*e for e in stain]
        total = sum(pow)
        temp = []
        for ele in stain:
            temp.append(ele/math.sqrt(total))
        normal_OD.append(temp)
    return np.asarray(normal_OD)

In [20]:

def convert_to_optical_densities(rgb,r0,g0,b0):
    OD = rgb.astype(float)
    OD[:,:,0] /= r0
    OD[:,:,1] /= g0
    OD[:,:,2] /= b0

    return -np.log(OD)

def color_deconvolution(rgb,r0,g0,b0,verbose=False):
    stain_OD = np.asarray([[0.18,0.20,0.08],[0.01,0.13,0.0166],[0.10,0.21,0.29]]) #hematoxylin, eosyn, DAB

    """
    n = []
    for r in stain_OD:
        n.append(r/norm(r))
    normalized_OD = np.asarray(n)
    """
    
    normalized_OD = normalize(stain_OD)
    
    #D = inv(normalized_OD)
    D = np.linalg.inv(normalized_OD)
    
    OD = convert_to_optical_densities(rgb,r0,g0,b0)

    ODmax = np.max(OD,axis=2)
    plt.figure()
    plt.imshow(ODmax>.1)

    # reshape image on row per pixel
    rOD = np.reshape(OD,(-1,3))
    # do the deconvolution
    rC = np.dot(rOD,D)

    #restore image shape
    C = np.reshape(rC,OD.shape)

    #remove problematic pixels from the the mask
    ODmax[np.isnan(C[:,:,0])] = 0
    ODmax[np.isnan(C[:,:,1])] = 0
    ODmax[np.isnan(C[:,:,2])] = 0
    ODmax[np.isinf(C[:,:,0])] = 0
    ODmax[np.isinf(C[:,:,1])] = 0
    ODmax[np.isinf(C[:,:,2])] = 0

    return (ODmax,C)

In [21]:
def supervised_deconvolve(imInput):
    # create stain to color map
    stain_color_map = htk.preprocessing.color_deconvolution.stain_color_map
    print('stain_color_map:', stain_color_map, sep='\n')

    # specify stains of input image
    stains = ['hematoxylin',  # nuclei stain
            'eosin',        # cytoplasm stain
            'null']         # set to null if input contains only two stains

    # create stain matrix
    W = np.array([stain_color_map[st] for st in stains]).T

    # perform standard color deconvolution
    imDeconvolved = htk.preprocessing.color_deconvolution.color_deconvolution(imInput, W)
    return imDeconvolved

In [22]:
supervised_time = []
images.sort()
for i in range(0,len(images)):
    inputImageFile = 'val/'+ images[i]
    imInput = skimage.io.imread(inputImageFile)[:, :, :3]
    t1= time.time()
    imDeconvolved =  supervised_deconvolve(imInput)
    supervised_time.append(time.time()-t1)
    im_c = cv2.hconcat([imDeconvolved.Stains[:, :, 0], imDeconvolved.Stains[:, :, 1]])
    cv2.imwrite("temp.jpg",im_c)
    im_dec = cv2.imread("temp.jpg")
    im_c = cv2.hconcat([imInput, im_dec])
    if i==0:
        final = copy.deepcopy(im_c)
    else:
        final = cv2.vconcat([final, im_c])
#cv2.imwrite("supervised.jpg", final)


stain_color_map:
{'hematoxylin': [0.65, 0.7, 0.29], 'eosin': [0.07, 0.99, 0.11], 'dab': [0.27, 0.57, 0.78], 'null': [0.0, 0.0, 0.0]}
stain_color_map:
{'hematoxylin': [0.65, 0.7, 0.29], 'eosin': [0.07, 0.99, 0.11], 'dab': [0.27, 0.57, 0.78], 'null': [0.0, 0.0, 0.0]}
stain_color_map:
{'hematoxylin': [0.65, 0.7, 0.29], 'eosin': [0.07, 0.99, 0.11], 'dab': [0.27, 0.57, 0.78], 'null': [0.0, 0.0, 0.0]}
stain_color_map:
{'hematoxylin': [0.65, 0.7, 0.29], 'eosin': [0.07, 0.99, 0.11], 'dab': [0.27, 0.57, 0.78], 'null': [0.0, 0.0, 0.0]}
stain_color_map:
{'hematoxylin': [0.65, 0.7, 0.29], 'eosin': [0.07, 0.99, 0.11], 'dab': [0.27, 0.57, 0.78], 'null': [0.0, 0.0, 0.0]}
stain_color_map:
{'hematoxylin': [0.65, 0.7, 0.29], 'eosin': [0.07, 0.99, 0.11], 'dab': [0.27, 0.57, 0.78], 'null': [0.0, 0.0, 0.0]}
stain_color_map:
{'hematoxylin': [0.65, 0.7, 0.29], 'eosin': [0.07, 0.99, 0.11], 'dab': [0.27, 0.57, 0.78], 'null': [0.0, 0.0, 0.0]}
stain_color_map:
{'hematoxylin': [0.65, 0.7, 0.29], 'eosin': [0.07, 0

In [23]:
sum(supervised_time)/len(images)

0.03678997357686361

***Approach-2***

In [ ]:

inputImageFile = 'val/'+ images[0]
imInput = skimage.io.imread(inputImageFile)[:, :, :3]

In [ ]:
imInput.shape

In [ ]:
plt.imshow(imInput)
_ = plt.title('Input Image', fontsize=16)


**Approach 2.2** - UnSupervised color deconvolution : Sparse non-negative matrix factorization

In [ ]:
# create initial stain matrix
W_init = W[:, :2]

# Compute stain matrix adaptively
sparsity_factor = 0.5

I_0 = 255
im_sda = htk.preprocessing.color_conversion.rgb_to_sda(imInput, I_0)
W_est = htk.preprocessing.color_deconvolution.separate_stains_xu_snmf(
    im_sda, W_init, sparsity_factor,
)

# perform sparse color deconvolution
imDeconvolved = htk.preprocessing.color_deconvolution.color_deconvolution(
    imInput,
    htk.preprocessing.color_deconvolution.complement_stain_matrix(W_est),
    I_0,
)

print('Estimated stain colors (in rows):', W_est.T, sep='\n')

# Display results
for i in 0, 1:
    plt.figure()
    plt.imshow(imDeconvolved.Stains[:, :, i])
    _ = plt.title(stains[i], fontsize=titlesize)

In [24]:
def unsupervised_matrix(imInput):
    # create initial stain matrix
    W_init = W[:, :2]

    # Compute stain matrix adaptively
    sparsity_factor = 0.5

    I_0 = 255
    im_sda = htk.preprocessing.color_conversion.rgb_to_sda(imInput, I_0)
    W_est = htk.preprocessing.color_deconvolution.separate_stains_xu_snmf(
        im_sda, W_init, sparsity_factor,
    )

    # perform sparse color deconvolution
    imDeconvolved = htk.preprocessing.color_deconvolution.color_deconvolution(
        imInput,
        htk.preprocessing.color_deconvolution.complement_stain_matrix(W_est),
        I_0,
    )

    print('Estimated stain colors (in rows):', W_est.T, sep='\n')
    return imDeconvolved

In [25]:
unsupervised_mtime=[]
images.sort()
print(images)
for i in tqdm(range(0,len(images))):
# for i in range(0,2):
    inputImageFile = 'val/'+ images[i]
    imInput = skimage.io.imread(inputImageFile)[:, :, :3]
    t1 = time.time()
    imDeconvolved =  unsupervised_matrix(imInput)
    unsupervised_mtime.append(time.time()-t1)
    im_c = cv2.hconcat([imDeconvolved.Stains[:, :, 0], imDeconvolved.Stains[:, :, 1]])
    cv2.imwrite("temp.jpg",im_c)
    im_dec = cv2.imread("temp.jpg")
    im_c = cv2.hconcat([imInput, im_dec])
    if i==0:
        final = copy.deepcopy(im_c)
    else:
        final = cv2.vconcat([final, im_c])
# cv2.imwrite("unsupervised_matrix.jpg", final)


['0_0.png', '0_1.png', '0_2.png', '0_3.png', '1_0.png', '1_1.png', '1_2.png', '1_3.png', '2_0.png', '2_1.png', '2_2.png', '2_3.png', '3_0.png', '3_1.png', '3_2.png', '3_3.png', '4_0.png', '4_1.png', '4_2.png', '4_3.png', '5_0.png', '5_1.png', '5_2.png', '5_3.png']


  4%|▍         | 1/24 [00:19<07:37, 19.90s/it]

Estimated stain colors (in rows):
[[5.48681410e-01 7.42603468e-01 3.84042706e-01]
 [4.23350189e-17 8.25525742e-01 5.64364465e-01]]


  8%|▊         | 2/24 [00:41<07:34, 20.65s/it]

Estimated stain colors (in rows):
[[5.40567008e-01 7.46728318e-01 3.87535972e-01]
 [4.41843977e-17 8.24381640e-01 5.66034373e-01]]


...

Estimated stain colors (in rows):
[[0.59311142 0.71818784 0.36390256]
 [0.03991185 0.8089975  0.58645553]]


 96%|█████████▌| 23/24 [07:43<00:18, 18.21s/it]

Estimated stain colors (in rows):
[[0.6000502  0.7156125  0.3575451 ]
 [0.14573878 0.7943007  0.58978522]]


100%|██████████| 24/24 [08:05<00:00, 20.22s/it]

Estimated stain colors (in rows):
[[0.59056089 0.71878426 0.36686105]
 [0.01612915 0.80376148 0.59473299]]


In [26]:
sum(unsupervised_mtime)/len(images)

20.184581408898037

**Approach 2.3** - UnSupervised color deconvolution : The PCA-based method of Macenko et al.

In [ ]:
w_est = htk.preprocessing.color_deconvolution.rgb_separate_stains_macenko_pca(imInput, I_0)

# Perform color deconvolution
deconv_result = htk.preprocessing.color_deconvolution.color_deconvolution(imInput, w_est, I_0)

print('Estimated stain colors (rows):', w_est.T[:2], sep='\n')

# Display results
for i in 0, 1:
    plt.figure()
    # Unlike SNMF, we're not guaranteed the order of the different stains.
    # find_stain_index guesses which one we want
    channel = htk.preprocessing.color_deconvolution.find_stain_index(
        stain_color_map[stains[i]], w_est)
    plt.imshow(deconv_result.Stains[:, :, channel])
    _ = plt.title(stains[i], fontsize=titlesize)

In [27]:
def unsupervised_pca(imInput):
    I_0 = 255
    w_est = htk.preprocessing.color_deconvolution.rgb_separate_stains_macenko_pca(imInput, I_0)
    # Perform color deconvolution
    deconv_result = htk.preprocessing.color_deconvolution.color_deconvolution(imInput, w_est, I_0)
    print('Estimated stain colors (rows):', w_est.T[:2], sep='\n')
    result=[]
    for i in 0, 1:
        channel = htk.preprocessing.color_deconvolution.find_stain_index(
        stain_color_map[stains[i]], w_est)
        result.append(deconv_result.Stains[:, :, channel])
    return result    

In [28]:
unsupervised_pcatime=[]
images.sort()
print(images)
for i in tqdm(range(0,len(images))):
# for i in range(0,2):
    inputImageFile = 'val/'+ images[i]
    imInput = skimage.io.imread(inputImageFile)[:, :, :3]
    t1 = time.time()
    imDeconvolved =  unsupervised_pca(imInput)
    unsupervised_pcatime.append(time.time()-t1)
    im_c = cv2.hconcat([imDeconvolved[0], imDeconvolved[1]])
    cv2.imwrite("temp.jpg",im_c)
    im_dec = cv2.imread("temp.jpg")
    im_c = cv2.hconcat([imInput, im_dec])
    if i==0:
        final = copy.deepcopy(im_c)
    else:
        final = cv2.vconcat([final, im_c])
# cv2.imwrite("unsupervised_pca.jpg", final)

['0_0.png', '0_1.png', '0_2.png', '0_3.png', '1_0.png', '1_1.png', '1_2.png', '1_3.png', '2_0.png', '2_1.png', '2_2.png', '2_3.png', '3_0.png', '3_1.png', '3_2.png', '3_3.png', '4_0.png', '4_1.png', '4_2.png', '4_3.png', '5_0.png', '5_1.png', '5_2.png', '5_3.png']


  4%|▍         | 1/24 [00:00<00:02,  8.33it/s]

Estimated stain colors (rows):
[[0.34600919 0.8127125  0.46880277]
 [0.59710088 0.71688105 0.35993346]]


  8%|▊         | 2/24 [00:00<00:02,  7.49it/s]

Estimated stain colors (rows):
[[0.33220341 0.81552628 0.47387529]
 [0.59379297 0.71907784 0.36102212]]
Estimated stain colors (rows):
[[0.36573217 0.81035407 0.45778407]
 [0.59351913 0.71876957 0.36208472]]


...

Estimated stain colors (rows):
[[0.31337963 0.80533062 0.5032254 ]
 [0.62988651 0.69820791 0.34021859]]


 96%|█████████▌| 23/24 [00:03<00:00,  7.31it/s]

Estimated stain colors (rows):
[[0.2319744  0.79746219 0.55699366]
 [0.63169887 0.69960105 0.33393848]]


100%|██████████| 24/24 [00:03<00:00,  6.97it/s]

Estimated stain colors (rows):
[[0.31086722 0.80416353 0.50663852]
 [0.62758485 0.69896171 0.34291367]]


In [29]:
sum(unsupervised_pcatime)/len(images)

0.11124852299690247